# **Filter Class**

In [0]:
import numpy as np

class Filter:
    
    ### constructor takes array which will be filter
    def __init__(self, filterM):
        self.filter = np.array(filterM)
        self.n = self.filter.shape[0]
        self.m = self.filter.shape[1]
        
    ### returns filter
    def get_filter(self):
        return self.filter
    
    ### change filter by delta matrix
    def add_delta(self, delta):
        self.filter += delta

    ### set the value of i,j element of the filter 
    def set(self, i, j, val):
        self.filter[i][j] = val

    ### returns the value of i,j element
    def get(self, i, j):
        return self.filter[i][j]

    ### takes matrix and returns filtered matrix
    def go_filter(self, data):
        resultdata = np.zeros((data.shape[0] - self.n + 1, data.shape[1] - self.m + 1))
        for i in range(data.shape[0] - self.n + 1):
            for j in range(data.shape[1] - self.m + 1):
                resultdata[i][j] = np.sum(self.filter * data[i:i+self.n,j:j+self.m])
        return resultdata
    
    ### takes matrix and poolfilter dimension, returns max-pooled matrix
    def go_maxpool(self, data, n, m):
        resultdata = np.zeros((int(data.shape[0] / n) , int(data.shape[1] / m)))
        for i in range(0, data.shape[0], n):
            for j in range(0, data.shape[1], m):
                resultdata[int(i/n)][int(j/m)] = np.max(data[i:i+n, j:j+m])
        return resultdata
    
    def go_maxpool(self,data,n,m):
        resultdata = np.zeros(data.shape[0] / n , data.shape[1] / m)
        for i in range(0, data.shape[0], n):
            for j in range(0,data.shape[1], m):
                resultdata[i/n][j/m] = data[i:i+n, j:j+m].mean()
        resultdata
        
        
    def avarage_pooling_backforward(self,data, loss_data, n, m):
        updated_data = np.zeros(data.shape)
        for i in range(0, data.shape[0], n):
            for j in range(0,data.shape[1], m):
                data[i: i + n, j: j + m] = (m*n)*resultdata[i/n][j/m]

    # pshiu, this method is pretty complex but in few words it updates convolutional filter
    # looking at loss matrix and pre-matrixes.
    def update_filter_one_matrix(self, data, loss_data):
        learning_rate = 0.0003
        updated_filter = np.zeros(self.filter.shape)
        updated_data = np.zeros(data.shape)
        for i in range(data.shape[0] + 1 - self.n):
            for j in range(data.shape[1] + 1 - self.m):
                # in summary resultdata[i][j] = self.filter * data[i:i+self.n, j : j+self.m]
                my_filter_part_data = data[i:i+self.n, j : j+self.m]
                my_data_part_data = updated_data[i:i+self.n, j : j+self.m]
                for x in range(my_filter_part_data.shape[0]):
                    for y in range(my_filter_part_data.shape[1]):
                        updated_filter[x][y] += loss_data[i][j]*my_filter_part_data[x][y]
                        # x derivative
                        my_data_part_data[x][y] +=loss_data[i][j]*self.filter[x][y]
                updated_data[i:i+self.n, j : j+self.m] = my_data_part_data
        self.filter =self.filter -  learning_rate*updated_filter
        return updated_filter, updated_data

# **Layer Class**

In [0]:

class Layer:
    ### constructor takes the list of filters 
    def __init__(self, num_filters):
        self.filters = []
        self.num_filters = num_filters
        self.matrixes = []
        self.preMatrixes = []
        self.n = 2
        self.m = 3
    
    ### returns the i-th filter
    def get_filter(self, i):
        return self.filters[i]
    
    ### return the list of filters
    def get_filters(self):
        return self.filters
    
    ### generates list of matrixes, from previous layer's matrixes and it's filters
    def gen_matrixes(self, preMatrixes):
        if len(self.filters) == 0:
            for x in range(len(preMatrixes)):
                each_filters = [Filter(np.random.randn(2,3)) for _ in range(self.num_filters)]
                self.filters.append(each_filters)
                
        self.preMatrixes = preMatrixes
        my_matrixes = [np.zeros((self.preMatrixes[0].shape[0] - self.n + 1, self.preMatrixes[0].shape[1] - self.m + 1)) for _ in range(self.num_filters)]

        for x in range(len(preMatrixes)):
            mat = preMatrixes[x]
            temp = 0
            for filt in self.filters[x]:
                my_matrixes[temp] += filt.go_filter(mat)
                temp+=1
        self.matrixes = my_matrixes
            
        
    ### returns i-th matrix of our matrixs' list
    def get_matrix(self, i):
        return self.matrix[i]
    
    ### returns list of matrixes
    def get_matrixes(self):
        return self.matrixes
    
    def layer_back_forward(self, loss_data_list):


        pre_matrix_list = self.preMatrixes
        updated_pre_matrix = [np.zeros(matr.shape) for matr in pre_matrix_list]
        
        updated_filters = []

        for x in range(len(self.filters)):
            each_filters = [np.zeros(matr.get_filter().shape) for matr in self.filters[x]]
            updated_filters.append(each_filters)
#         print(self.filters)
        for x in range(len(self.filters)):
            filt = self.filters[x]
            for index in range(len(filt)):
                cur_filt = filt[index]
                cur_loss_data = loss_data_list[index]
                for y in range(len(pre_matrix_list)):
                    mat = pre_matrix_list[y]
                    updated_filter, updated_data = cur_filt.update_filter_one_matrix(mat, cur_loss_data)
                    updated_filters[x][index]+=updated_filter
                    updated_pre_matrix[y]+=updated_data
        return updated_pre_matrix
           
    


# CNN Class

In [0]:
class CNN:
    ### constructor takes the starting matrix and filters list for each layer
    ### layerFilters is 4D list
    ### a = CNN(ar, )
    ## update.
    ## layerFilters listshi mewera listi filtrebisa layerebis mixedvit
    ## exla shignit ubralod ricxvs chavwer tu ramdeni filtri unda mas.
    def __init__(self, stMatrix, layerFilters):
        self.stMatrix = stMatrix
        self.layers = []
        for layerFilter in layerFilters:
#             print(layerFilter)
            self.layers.append(Layer(layerFilter))
    
    ### returns all layer objects 
    def get_layers(self):
        return self.layers
    def set_x(self,X):
        self.stMatrix = X
    
    ### returns i-th layer
    def get_layer(self, i):
        return self.layers[i]
        
    ### returns flattened np-array for fully connection NL
    def forward_prop(self):
        matrixList = [self.stMatrix]
        for layer in self.layers:
            layer.gen_matrixes(matrixList)
            matrixList = layer.get_matrixes()
        return self.flatten(matrixList)
            
    ### static method, flattens 3D array
    def flatten(self, matrixList):

        lst = []
        for matrix in matrixList:
            lst+=(matrix.flatten().tolist())
        return lst
    
    def unflatten(self, flatten_matrix):
        my_matrix = self.layers[len(self.layers)-1].get_matrixes()
        my_loss_matrix = [np.zeros(tmp.shape) for tmp in my_matrix]
        temp = 0
        for index in range(len(my_loss_matrix)):
            mat = my_loss_matrix[index]
            for x in range(mat.shape[0]):
                for y in range(mat.shape[1]):
                    my_loss_matrix[index][x][y] = flatten_matrix[0][temp]
                    temp += 1
        return my_loss_matrix
        
    def backward(self, delta_x):
        new_delta_x = delta_x
        for layer in reversed(self.layers):
            #WINEBIS listi da loss_datas listi
            new_delta_x = layer.layer_back_forward(new_delta_x)
    

# Dense Class

In [0]:
class Dense:
    
    ### constructor takes parameter's array(flattened array) and answer array(1x10)
    def __init__(self, X, Y):
        self.i=0
        self.X = X
        self.Y = Y
        self.cachedLoss = []
    
    # just return e^(-z)
    def my_exp(self,z):
        return np.exp(-z)
    
    def set_y(self, Y):
        self.Y=Y
    
    def set_matrix(self,mat):
        if self.X is None:
            self.W0 = np.random.randn(mat.size, 64)/100
            self.W1 = np.random.randn(64, 10)/100
        self.X = mat
        
        
        # our zigmoid function
    def zigmoid_function(self,z):
        return 1/(1 + self.my_exp(z))
    
    def costfn_class(self, X, y):
#         X = self.zigmoid_function(X)
        error= -1*y*np.log(X) + (y-1)*np.log(1-X)
#         my_sum = np.sum(abs(error))
        return error
    
    def cross_entropy(self, predictions, targets, epsilon=1e-10):
        predictions = np.clip(predictions, epsilon, 1. - epsilon)
        ce = targets*np.log(predictions+1e-9) + (targets - 1)*np.log(1-predictions + 1e-9)
        return ce
    
    
        # ferivative of our zigmoid function
    def derivative_zigmoid(self,z):
        return self.my_exp(z) / np.power((1 + self.my_exp(z)), 2)
    
    # our fucntion doing feed forward alghoritm.
    def feed_forward(self,X, W0, W1):
        h0 = np.dot(X, W0)  # our first matrix after using first hidden layer W0
    #     h0 = zigmoid_function(h0)  # our first matrix using zigmoid.
        
        h3 = np.dot(h0, W1) # our result after using second hidden layer W1
        h1 = self.zigmoid_function(h3) # our result matrix after using zigmoid.
        return h0, h1 # return result.
    
    # compute our error which is h1-y but we also take into consideration the zigmoid function.
    def my_error(self,h1):
#         error = h1 - self.Y
#         return error
        return self.cross_entropy(h1,self.Y)


    def backward(self,X, W0, W1, h1, h0, alfa_rate):
        # firstly lets compute error h1-y
        error = self.my_error(h1)
#         if self.cachedLoss != []:
#             error = error*0.1 + 0.9*self.cachedLoss
#         self.cachedLoss = error    
        
        bla_W1 = W1
        # first lets update W1 which is far easier W1 = (h0)T *(h1 - Y)
        transpose_h0 = np.transpose(h0)
        error = error * self.derivative_zigmoid(h1)

        # now lets update W0 too which is bit complex.
        # W0 = XT.(ERROR).W1T
        transpose_X = np.transpose(X)
        transpose_W1 = np.transpose(bla_W1)


        first_step = np.dot(error, transpose_W1)
        second_step = np.dot(transpose_X, first_step)
        delta_x = np.dot(error, np.transpose(np.dot(self.W0, self.W1)))
        W0 -= second_step*alfa_rate
        W1 -= np.dot (transpose_h0, error)*alfa_rate
        return delta_x
    

    def predict(self, back):
        h0, h1 = self.feed_forward(self.X, self.W0, self.W1)
        delta_x = None
        if back:
            delta_x = self.backward(self.X, self.W0, self.W1, h1, h0, 0.0005)
        return delta_x, h1

# Main Model Class

In [0]:
class Final:
    def __init__(self, cnnOBJ, denseOBJ):
        self.cnn = cnnOBJ
        self.dense = denseOBJ
        self.letters = ['ე', 'ი', 'კ', 'ჟ', 'რ', 'ტ', 'ფ', 'ქ', 'ყ', 'ჰ']
        
    def find_max_ind(arr):
      index = 0
      max_num = arr[0][0]
      for i in range(len(arr[0])):
          if arr[0][i] > max_num:
              max_num = arr[0][i]
              index = i
      return index
        
    def go(self, img):
        h = go_forest(img, [[1,0,0,0,0,0,0,0,0,0]], True, False)
        ind = find_max_ind(h)
        return self.letters[ind]
    
    def go_forest(X, Y, debug=False, back = True):
        self.cnn.set_x(X)
        self.dense.set_y(Y)
        for _ in range(1):
            h1 = predict(self.cnn, self.dense, back)
        return h1
    
    def predict(cnn, dense_class, back):
        flatten_array = np.array(cnn.forward_prop())
        flatten_array = np.reshape(np.array(flatten_array), (flatten_array.shape[0],1))

        #now lets start with dense class to work with neural network.
        flatten_array = np.transpose(flatten_array)

        # dense_class = Dense(np.transpose(flatten_array),Y)
        dense_class.set_matrix(flatten_array)
        delta_x, h1 = dense_class.predict(back)

        if back:
            #listi maq matricebis
            delta_x = cnn.unflatten(delta_x)
            cnn.backward(delta_x)

        return h1


# Executable

In [0]:
import os
from PIL import Image, ImageOps
import numpy as np
import csv
from scipy.misc import imsave

input_folder = './input/ე' #needs to be filled

def cutImg(path, rotate, size):
    im = Image.open(path)
    rgb_im = im.convert('RGB')
    arr = np.array(rgb_im)

    startTop = 0
    endBot = arr.shape[1]
    letterFound = False

    for i in range(arr.shape[0]):
        isLetter = False
        for j in range(arr.shape[1]):
            if arr[i][j][0] <= 210 and arr[i][j][1] <= 210 and arr[i][j][2] <= 210:
                isLetter = True
                break

        if isLetter and not letterFound:
            letterFound = True
            startTop = i

        if not isLetter and letterFound:
            letterFound = False
            endBot = i

        letterFound = isLetter

    cropped = rgb_im.crop((0, startTop, arr.shape[1], endBot))

    if rotate == 1:
        arr1 = np.array(cropped)
        arr1 = np.rot90(arr1, axes=(0, -2))
        cropped = Image.fromarray(arr1)
    elif rotate == -1:
        arr1 = np.array(cropped)
        arr1 = np.rot90(arr1, axes=(-2, 0))
        cropped = Image.fromarray(arr1)

    if size != 0:
        cropped = cropped.resize((size, size), resample=Image.BILINEAR)

    cropped.save(path)


def cropImg(path):
    cutImg(path, 1, 0)
    cutImg(path, -1, 64)

def add_border(input_image, border, color=0):
    img = Image.open(input_image)
    if isinstance(border, int) or isinstance(border, tuple):
        bimg = ImageOps.expand(img, border=border, fill=color)
    else:
        raise RuntimeError('Border is not an integer or tuple!')
    return bimg
    
def binarize_array(numpy_array, threshold=180):
    for i in range(len(numpy_array)):
        for j in range(len(numpy_array[0])):
            if numpy_array[i][j] > threshold:
                numpy_array[i][j] = 255
            else:
                numpy_array[i][j] = 0
    return numpy_array

def convert_bw(img):
    image = img.convert('L')  # convert image to monochrome
    image_arr = np.array(image)
    image_arr = binarize_array(image_arr)
    return image_arr

def format_picture(image_path):
    img_borders = add_border(image_path, border=10, color='white')

    img_convert = img_borders.convert("RGBA")
    fff = Image.new('RGBA', img_convert.size, (255,)*4)
    out = Image.composite(img_convert, fff, img_convert)
    out = out.convert(img_borders.mode)
    
    img_bw = convert_bw(out)
    img_name = "{}.{}".format('test', 'jpg')
    save_path = "{}/{}".format(input_folder, img_name)

    imsave(save_path, img_bw)
    img = Image.open(save_path)
    img = img.resize((40, 40), resample=Image.BILINEAR)
    img.save(save_path)

    imsave("{}".format(save_path), img)
    cropImg("{}".format(save_path))
    image = Image.open(save_path)
    image = image.resize((40,40), resample = Image.BILINEAR)

    return convert_bw(image)

def generate_ans(image):
    arr = np.asarray(image)
    return arr


import pickle  
if __name__ == "__main__":
  
    filehandler = open("savedModel1.pkl", 'rb') 
    model1 = pickle.load(filehandler)
  
    image_files = os.listdir(input_folder)
    result = []
    for index in range(len(image_files)):
        image_array = format_picture('{}/{}'.format(input_folder, image_files[index]))
        image_array = image_array/255
        print(image_array.shape)
        ans = model1.go(image_array)
        result += [[str(index), ans]]


    filename = 'result.csv'
    with open(filename, 'w') as f:
        csv.writer(f).writerows(result)